In [ ]:
import anywidget
import ipywidgets as ipw
import traitlets
from ipymlwidgets import Image, ImageAnnotated
from ipymlwidgets.traits import SupportedTensor
import torch

from ipymlwidgets.widgets.ocr import ItemOCR

# Test the ItemOCR widget
image_tensor = torch.rand(3, 20, 30)
text_content = "This is a sample OCR text that demonstrates the ItemOCR widget functionality."

height = 100
gradient = torch.linspace(0, 1, height).unsqueeze(1).expand(height, 30)    
image_tensor = gradient.unsqueeze(0).expand(3, height, 30)


item_ocr = ItemOCR(
    image=image_tensor,
    text=text_content,
    max_height="20px"
)

item_ocr


In [ ]:
from ipymlwidgets import Canvas
import torch
height = 100
gradient = torch.linspace(0, 1, height).unsqueeze(1).expand(height, 30)    
image_tensor = gradient.unsqueeze(0).expand(3, height, 30)

canvas = Canvas(layers=2)

canvas







In [ ]:
from ipymlwidgets import Image, ImageAnnotated
import torch
import ipywidgets as W
size = (320,320)

xy = torch.randint(size[0]-10, (1,2))
boxes = torch.cat([xy, xy + 10], dim=-1).float()
image = torch.rand(3,*size)
#image = Image(image)
image = ImageAnnotated(image, boxes=boxes)
display(image)


In [ ]:
import anywidget
import traitlets
import ipywidgets as widgets

class ChildWidget(anywidget.AnyWidget):
    """A red box widget.

    Returns:
        None: This widget displays a red box.
    """
    _esm = """
    function render({ el }) {
        const box = document.createElement("div");
        box.style.width = "100px";
        box.style.height = "100px";
        box.style.background = "red";
        el.appendChild(box);
    }
    export default { render };
    """

class ParentWidget(anywidget.AnyWidget):
    """A blue padded box containing a child widget.

    Args:
        child (widgets.Widget): The child widget to display inside the blue box. Defaults to None.

    Returns:
        None: This widget displays a blue box containing the child widget.
    """
    _esm = """
    function render({ model, el }) {
        el.innerHTML = "";
        const box = document.createElement("div");
        box.style.background = "blue";
        box.style.padding = "20px";
        box.style.display = "inline-block";
        
        // Get the child widget model
        const childModel = model.get("child");
        console.log(childModel);
        if (childModel) {
            // Create a container for the child widget
            const childContainer = document.createElement("div");
            box.appendChild(childContainer);
            
            // The child widget will automatically render itself
            // We need to tell Jupyter to render the child widget in our container
            childModel.widget_manager.display_model(childModel, { el: childContainer });
        }
        
        el.appendChild(box);
    }
    export default { render };
    """
    child = traitlets.Instance(widgets.Widget, allow_none=True).tag(sync=True, **widgets.widget_serialization)

# Usage
child = ChildWidget()
parent = ParentWidget(child=child)
display(parent)





In [ ]:
import anywidget
import traitlets
import ipywidgets as ipw

class Box(anywidget.AnyWidget):
    _esm = """
    async function unpack_models(model_ids, manager) {
        return Promise.all(
            model_ids.map(id => manager.get_model(id.slice("IPY_MODEL_".length)))
        );
    }
    async function render(view) {
       let model_ids = view.model.get("children"); // ["IPY_MODEL_<model_id>", ...]
        let children_models = await unpack_models(model_ids, view.model.widget_manager);
        for (let model of children_models) {
            let child_view = await view.create_child_view(model);
            view.el.appendChild(child_view.el);
        }
    }
    export default { render };
    """
    children = traitlets.List(trait=traitlets.Instance(ipw.DOMWidget)).tag(sync=True, **ipw.widget_serialization)


box = Box(children=[
    ipw.IntProgress(),
    ipw.IntSlider(),
])

box

In [ ]:
import anywidget
import traitlets

class CounterWidget(anywidget.AnyWidget):
    _esm = """
    function render({ model, el }) {
      let button = document.createElement("button");
      button.innerHTML = `count is ${model.get("value")}`;
      button.addEventListener("click", () => {
        model.set("value", model.get("value") + 1);
        model.save_changes();
      });
      model.on("change:value", () => {
        button.innerHTML = `count is ${model.get("value")}`;
      });
      el.classList.add("counter-widget");
      el.appendChild(button);
    }
    export default { render };
    """
    _css = """
    .counter-widget button { color: white; font-size: 1.75rem; background-color: #ea580c; padding: 0.5rem 1rem; border: none; border-radius: 0.25rem; }
    .counter-widget button:hover { background-color: #9a3412; }
    """
    value = traitlets.Int(0).tag(sync=True)

CounterWidget(value=42)

In [ ]:
import anywidget
import ipywidgets as ipw
import traitlets

from ipymlwidgets import Image, ImageAnnotated
import torch
import ipywidgets as W
size = (320,320)

class CounterWidget(anywidget.AnyWidget):
    _esm = """
    function render({ model, el }) {
      let button = document.createElement("button");
      button.innerHTML = `count is ${model.get("value")}`;
      button.addEventListener("click", () => {
        model.set("value", model.get("value") + 1);
        model.save_changes();
      });
      model.on("change:value", () => {
        button.innerHTML = `count is ${model.get("value")}`;
      });
      el.classList.add("counter-widget");
      el.appendChild(button);
    }
    export default { render };
    """
    _css = """
    .counter-widget button { color: white; font-size: 1.75rem; background-color: #ea580c; padding: 0.5rem 1rem; border: none; border-radius: 0.25rem; }
    .counter-widget button:hover { background-color: #9a3412; }
    """
    value = traitlets.Int(0).tag(sync=True)


class Box(anywidget.AnyWidget):
    _esm = """
    async function unpack_models(model_ids, manager) {
        return Promise.all(
            model_ids.map(id => manager.get_model(id.slice("IPY_MODEL_".length)))
        );
    }
    async function render({ model, el }) {
       let model_ids = model.get("children");
        let children_models = await unpack_models(model_ids, model.widget_manager);
        for (let model of children_models) {
            let child_view = await model.widget_manager.create_view(model);
            el.appendChild(child_view.el);
        }
    }
    export default { render };
    """
    children = traitlets.List(trait=traitlets.Instance(ipw.DOMWidget)).tag(sync=True, **ipw.widget_serialization)

image = torch.rand(3,*size)
image = ImageAnnotated(image)

#image.observe(print, "mouse_enter")
text_widget = ipw.Text(
    value="Hello, world!",
    disabled=False,  # Make it read-only
    layout=ipw.Layout(width='auto')
)

Box(children=[image, text_widget])

In [ ]:
from ipymlwidgets.widgets import Box, Image
from ipymlwidgets.traits import SupportedTensor
import ipywidgets as W
import torch 

class ItemOCR(Box):

    def __init__(self, image: SupportedTensor, text: str):
        self._image = Image(image)
        self._text = W.Label(value=text)
        super().__init__(children=[self._image, self._text])



# box = Box(children=[W.Label(value="Hello, world!")])

item = ItemOCR(image=torch.rand(3,120,120), text="Hello, world!")

display(item)